In [44]:
import requests
import copy
import music21 as m21
from music21 import *
from pathlib import Path
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import json
import tensorflow.keras as keras


from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical

import pickle as pkl
import joblib



import os
import glob 
from pathlib import Path

import configure

def load_songs(data_path):
    """loads all the songs in the data folder and 
    converts them into a m21 stream object
    args: data path
    returns: a list of all the converted songs
    Important concept: parsing is the process of recognizing and identifying the components of
    a particular input"""
    songs = []
    for path, subdirs, files in os.walk(data_path):
        for i, file in enumerate(files):
            try:
                if file[-3:] == "mid" or file[-4:] == "midi":
                    song = m21.converter.parse(os.path.join(path, file)) #parsing...crea un objeto stream.Score
                    songs.append(song)
            except:
                print("Failed loading the: {} song".format(i))
    
    print("{} songs successfully loaded and converted to m21 stream objects".format(len(songs)))

    return songs


def has_acceptable_duration(song, acceptable_durations):
    """Returns a boolean for checking if the song copmponents has
    all its elements of the acceptable durations
    args:
    song: m21 stream
    acceptable_durations: list cointaining the acceptable durations
    Al decir for note in song.flat.notesAndRests repasamos toda la cancion"""
    #load the song: reads it as argument


    #check for each component if has acceptable durations
    for note in song.flat.notesAndRests:
        if note.duration.quarterLength not in acceptable_durations:
            return False
    #returns True/False
    return True
    
def songs_has_no_chords(song):
    """Returns a boolean True indicating if song has no chords"""
    for element in song.flat.notesAndRests:
        if isinstance(element, m21.chord.Chord):
            return False
    #Returns False by default
    return True


def check_durations(song, acceptable_durations):
    """Returns a boolean for checking if the song copmponents has
    all its elements of the acceptable durations
    args:
    song: m21 stream
    acceptable_durations: list cointaining the acceptable durations
    """
    #load the song: reads it as argument


    #check for each component if has acceptable durations
    durations = []
    for note in song.flat.notesAndRests:
        durations.append(note.duration.quarterLength) 
            
    return durations

def aproximate_duration(duration, acceptable_durations):
    """Aproximates the non-accep´table durations depending on
    the duration of the element:
    if < 0.25: round to 0.25
    if > 0.25 and is multiple of 0.25 ---> round to the highest acceptable duration
    if > 0.25 and is not multiple of 0.25 ---> round to the nearest multiple inside the list"""

    #if number less than 0.25 ---> aproximate to 0.25
    if duration < 0.25:
        return 0.25

    else:
        #If is multiple of 0.25 but higher than the highest in duration list reduce to the highest
        if duration % 0.25 == 0:
            return acceptable_durations[-1]
            
    
        else: #Round up to the nearest multiple of 0.25
            return 0.25 * round(duration / 0.25)

def repair_durations(song, acceptable_durations):
    for note in song.flat.notesAndRests:
        if note.duration.quarterLength not in acceptable_durations:
            note.duration.quarterLength  = aproximate_duration(note.duration.quarterLength, acceptable_durations)
    return song
    
            

def transpose_song(song):
    """Transpose the song to Cmajor/Aminor
    arg: song as ms21 object
    return: song transposed"""
    #Get the original key of the song
    parts = song.getElementsByClass(m21.stream.Part) #Extrae todas las partes de la canción (violin, viola, etc)
    measures_part0 = parts[0].getElementsByClass(m21.stream.Measure) #Extrae los elementos de la parte0 como referencia
    
    try: 
        key = measures_part0[0][4] ##tomo la primera parte de measures0 y extraigo de esa lista el elemento 4 que es key

    except:
        key = song.analyze("key") #si no resulta de esa forma que intente este metodo

    #If we cant get the key by the previous method because is not in the song, estimate it
    if not isinstance(key, m21.key.Key): #if the song doesnt hace any key stored
        key = song.analyze("key") #estimate it...

    #Calculate the interval or distance to transpose
    #si esta en tono mayor calcula intervalo con A minor
    #print("The song is originilally in the key of {}".format(key))
    if key.mode == "minor":
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("A")) #key.tonic da el tono en que está

    elif key.mode == "major":
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("C")) #key.tonic da el tono en que está

    #Transpose the song

    transposed_song = song.transpose(interval)

    #print("The song has been transposed to the key of {}".format(transposed_song.analyze("key")))

    return transposed_song


def encode_song(song, time_step = 0.25):
    """method for converting a stream object into a time series sequence,
    considering a time step of 0.25 (1 semicorchea). Le time series avanzara a
    un paso de 1 semicorchea
    returns: a list with the form [60,_,_, r, 67,_, 74, _, _, 34]"""
    #song = song.chordify() ##testing
    encoded_song = []
    chord_count = 0
    for event in song.flat.notesAndRests: #crea una lista de todos los elementos de la cancion (notas, rests)
        #si event es una nota guarda la nota
        if isinstance(event, m21.note.Note):
            symbol = event.pitch.midi #le asigna su equvalente de la nota en valor midi
        #Si es un acorde...
        elif isinstance(event, m21.chord.Chord):
           #testing what happens if not considering he chords
            current_chord= ".".join(str(n.pitch.midi) for n in event)
            symbol = current_chord

            continue
        elif isinstance(event, m21.note.Rest):
            symbol = "r"
    
        #Calcula el nro de time_steps que dura el evento:
        nr_time_steps = int(event.duration.quarterLength / time_step)

        #Ahora voy guardando en encoded song considerando que si estoy al principio (Nr_time_step = 0)
        #append la nota/rest y para el resto "_"

        for step in range(nr_time_steps):
            if step == 0:
                encoded_song.append(symbol)
            else:
                encoded_song.append("_")

    #Convierto con map todos los caracteres de encoded_song a str
    #y luego los uno separados por un " "
    encoded_song = " ".join(map(str, encoded_song))
    return encoded_song



#Saving all the encoded songs in one file
SEQUENCE_LENGTH = 64 #nr of repetitions of "/"
SINGLE_FILE_PATH = "single_dataset" #name of the single file to be created 

def load(dataset_path):
    """Utility for reading individual songs from a directory"""
    with open(dataset_path,"r") as fp:
        song = fp.read()
        return song

def create_single_file_dataset(dataset_path, single_file_path, sequence_length):
    """Se crea un gran archivo tipo strong donde se almacenan todas las canciones del dataset,
    separadas por un delimitador /
    Delimitador: simbolo "/ " repetido 64 veces, ya que asi las leen las LSTM
    args:
    dataset_path: path of the directory of individual songs (the already encoded songs)
    single_file_path: where the single file to bre created will be saved/name of the single file
    sequence_length: to be used for indicating the beginning of a new song"""
    songs = " "
    new_song_delimiter = "/ " * sequence_length #separador de canciones
    print("Creating single file sequence...")
    #Paso por todos los archivos del directorio dataset_path, load song, put delimiters 
    for path, _, files,  in os.walk(dataset_path):
        for file in files:
            file_path = os.path.join(path, file) #ubicacion exacta de la cancion
            song = load(file_path) #metodo para load la cancion
            songs = songs + song + " " + new_song_delimiter

    songs = songs[:-1] #recorto espacio que quedaria en el deliminador de la ultima cancion

    #Save the songs
    with open(single_file_path, "w") as fp:
        fp.write(songs)
    print("Single sequence file created...")
    return songs


def preprocess(dataset_path):
    """sequence of reading, processing the midi files into encoded songs and
    saving them into a new SAVE_DIR
    args:
    dataset_path: original midi data folder
    Steps:
    #read the midi songs from the data_path and convert them into stream m21 (saved in a list)
    #Correct each song with acceptable durations criteria
    #Transpose each song to Cmajor/Aminor key
    #encode each song into its equivalent midi numbers and rest symbols
    #Save all songs in SAVE_DIR
    
    """
    #Empty the save_dir directory before starts...
    [f.unlink() for f in Path(configure.SAVE_DIR).glob("*") if f.is_file()] 

    #Load and convert songs to m21 streams...
    print("Loading songs...")
    initial_songs = load_songs(dataset_path)
    
    #Correct each song with acceptable durations criteria
    songs = []
    for song in initial_songs:
        song = repair_durations(song, configure.ACCEPTABLE_DURATIONS)
        songs.append(song)
    print("{} songs loaded and with their duration corrected".format(len(songs)))

    #Transpose and encode each song...
    print("Transposing and encoding  songs...")
    out_songs = 0
    enc_songs = 0
    for i, song in enumerate(songs):
        song = transpose_song(song)
        encoded_song = encode_song(song, time_step= 0.25)
        enc_songs += 1
        #Save sons as text file in SAVE_DIR 
        save_path = os.path.join(configure.SAVE_DIR, str(i)) #saves each song with a number
        with open(save_path, "w") as fp:
            fp.write(encoded_song)
    print("Number of encoded songs saved in Save_dir:", enc_songs)

#Create a dictionary for mapping the symbols

def create_mapping(songs, mapping_json_name):
    """Creates a dictionary of each symbol: integer
    args:
    songs: single file with all songs encoded together
    mapping_json_name: name that will have the dictionary
    
    returns:
    Saves the vocabulary as mappins_json_name json file
    Vocabulary_elements (list): list of all the unique elements"""
    mappings = {}
    songs_elements = songs.split() #separa todos los elementos del archivo songs
    vocabulary_elements = list(set(songs_elements)) #lista de los elementos unicos
    for i, symbol in enumerate(vocabulary_elements):
        mappings[symbol] = i

    with open(mapping_json_name, "w") as fp:
        json.dump(mappings, fp, indent = 4)
    #print("Vocabulary:", vocabulary)
    print("VOCAB_LENGTH:", len(vocabulary_elements))
    print("Mapping created...")
    return vocabulary_elements
    





#Convert  the single file symbols into integers using the mapping
def convert_into_integers(single_file):
    """Takes the single file and coverts its symbols into integers using
    the mapping created"""
    int_single_file =[] #vaciamos el mapeo a una lista
    #Open the json mapping file
    with open (configure.MAPPING_JSON_NAME, "r") as fp:
        mappings = json.load(fp)
    
    #Split of elements in single file
    single_file = single_file.split()

    #Map songs into integers
    for symbol in single_file:
        int_single_file.append(mappings[symbol])
    print("Single file converted to integers using the mapping")
    return int_single_file

#Create training sequences 
#Generating training sequences...
#las LSTM se estructuran tomando una secuencia de notas y prediciendo cual es la proxima
#Por ser supervisado, se le da una secuencia y se le muestra un target; asi se va entrenando
#Por ello tomaremos una secuencia de 64 time_steps (que equivalen a 4 compases de 4/4) como sample
#y como target le mostramos la siguiente nota o figura. Recuerda que cada time_step es una semicorchea
#Para ello las secuencias se construyen considerando que se trata de un time series, mviendose
#con un window hacia adelante
#En este caso, dado que tenemos un sequence length de 64 timesteps, si hay 100 symbols en total
#y nos movemos de a uno en la ventana, tendriamos un total de secuencias de 100 - 64

def generate_training_sequences(sequence_length):
    """Takes the integer converted sequence and creates sequences of examples and targets:
    examples: 64 elements
    target: the following element
    output: inputs and targets
    input vector shape: nr_of_songs x sequence_length x nr_of_symbols(or features)"""
    #load the songs and map them to int
    songs = load(SINGLE_FILE_PATH)
    int_song = convert_into_integers(songs)

    inputs = [] #to save the examples/sequences
    targets = []
    number_of_sequences = len(int_song) - sequence_length # cantidad de secuencias que se van a generar

    for i in range(number_of_sequences):
        inputs.append(int_song[i: i + sequence_length])
        targets.append(int_song[i + sequence_length]) #la siguiente nota/rest
    
    #Convert to one-hot encoding for creating the input vectors
    vocab_size = len(set(int_song)) #unique elements
    print("Vocab size:", vocab_size)
    inputs = keras.utils.to_categorical(inputs, num_classes = vocab_size)
    #Convert targets to array
    targets = np.array(targets)
    print("Training data successfully generated")
    return inputs, targets, vocab_size


###PP


def create_data_sequences(mapped_song):
    """ Creates the training data taking the sequence
    and creating sequences of 100 elements as input
    and the next element as the targets, moving in a window
    of 1 step. Also shapes the input data to the format
    demanded by the LSTM: (len_dataset, SEQUENCE_LENGTH,1)
    args: 
    inputs:mapped song: list of elements mapped into integers
    returns: 
    - input data before scaling and reshaping (list)
    - input_data_final: scaled and reshaped data ready for training (array)
    - target data"""

    SEQUENCE_LENGTH = 100 #largo de cada secuencia
    NUM_SEQUENCES = len(mapped_song) - SEQUENCE_LENGTH #total secuencias

    input_data = []
    targets = []
    #Creating the sequences...
    for i in range(0, NUM_SEQUENCES, 1):
        input_data.append(mapped_song[i: i + SEQUENCE_LENGTH])
        targets.append(mapped_song[i + SEQUENCE_LENGTH])

    print("Training data created")
    print("Dataset size:", len(input_data))
    #input_data = np.array(input_data) #input data before scaling
    targets = np.array(targets)
    #Normalize input data
    scaler = MinMaxScaler()
    input_data_scaled = scaler.fit_transform(input_data)
    
    #Reshape the input into a format compatible with LSTM layers...
    input_data_final = np.reshape(input_data_scaled, ((len(input_data_scaled), SEQUENCE_LENGTH,1)))
    
    #input_data = input_data / len(set(mapped_song))

    #One hot encode the output
    targets = to_categorical(targets)

    return input_data , input_data_final, targets

#CHECKING AND LEARNING ABOUT DATA

###pp
def mapping_data(song):
    """Mapping all the elements of song to integers and
    saves the mappings dictionary
    arg:
    input (list): song encoded with the initial form (B4,r, F)
    output (list): mapped_song, list with song mapped to integers"""
    #obtener vocabulario los elementos unicos
    vocabulary = list(set(song))
    vocab_length = len(vocabulary)
    print("vocab length:", vocab_length)
    #crear un diccionario {elemento: numero}
    mapping_dict = {}
    for i, element in enumerate(vocabulary):
        mapping_dict[element] = i

    #Save mapping dict
    with open("mapping_modular.json", "w") as fp:
        json.dump(mapping_dict, fp, indent = 4)

    #mapear la cancion
    mapped_song = []
    for element in song:
        mapped_song.append(mapping_dict[element])
    #return cancion mapeada
    return mapped_song, vocab_length, vocabulary
  
###ppp
def create_data_sequences(mapped_song):
    """ Creates the training data taking the sequence
    and creating sequences of 100 elements as input
    and the next element as the targets, moving in a window
    of 1 step. Also shapes the input data to the format
    demanded by the LSTM: (len_dataset, SEQUENCE_LENGTH,1)
    args: 
    inputs:mapped song: list of elements mapped into integers
    returns: 
    - input data before scaling and reshaping (list)
    - input_data_final: scaled and reshaped data ready for training (array)
    - target data"""

    SEQUENCE_LENGTH = 100 #largo de cada secuencia
    NUM_SEQUENCES = len(mapped_song) - SEQUENCE_LENGTH #total secuencias

    input_data = []
    targets = []
    #Creating the sequences...
    for i in range(0, NUM_SEQUENCES, 1):
        input_data.append(mapped_song[i: i + SEQUENCE_LENGTH])
        targets.append(mapped_song[i + SEQUENCE_LENGTH])

    print("Training data created")
    print("Dataset size:", len(input_data))
    #input_data = np.array(input_data) #input data before scaling
    targets = np.array(targets)
    #Normalize input data
    scaler = MinMaxScaler()
    input_data_scaled = scaler.fit_transform(input_data)
    
    #Reshape the input into a format compatible with LSTM layers...
    input_data_final = np.reshape(input_data_scaled, ((len(input_data_scaled), SEQUENCE_LENGTH,1)))
    
    #input_data = input_data / len(set(mapped_song))

    #One hot encode the output
    targets = to_categorical(targets)

    return input_data , input_data_final, targets
   


In [39]:
if __name__=="__main__":
    #Load, correct duration, transpose, encode and save to a save_dir...
    preprocess(configure.DATA_PATH)

    #Create a single string of all the songs delimited by ///
    print("Creating the single file...")
    single_file_song =  create_single_file_dataset(configure.SAVE_DIR,configure.SINGLE_FILE_PATH, configure.SEQUENCE_LENGTH)
    print("Single file created")
    #Mapping all the symbols into a integer vocabulary
    print("Mapping...")
    # mapped_song, vocab_length = mapping_data(single_file_song)
    create_mapping(single_file_song, configure.MAPPING_JSON_NAME)

Loading songs...
Failed loading the: 80 song
Failed loading the: 122 song
Failed loading the: 160 song
464 songs successfully loaded and converted to m21 stream objects
464 songs loaded and with their duration corrected
Transposing and encoding  songs...
Number of encoded songs saved in Save_dir: 464
Creating the single file...
Creating single file sequence...
Single sequence file created...
Single file created
Mapping...
VOCAB_LENGTH: 78
Mapping created...


In [40]:
mapped_song, vocab_length, vocabulary = mapping_data(single_file_song)  

vocab length: 14


In [41]:
#Creates the vocabulary in a json mapping file
vocabulary_elements = create_mapping(single_file_song, configure.MAPPING_JSON_NAME)
#Mappinbg the single file into integers using mappings vocabulary
single_file_integer = convert_into_integers(single_file_song)
#Generate training sequences
X, X_f, y = create_data_sequences(single_file_integer)


VOCAB_LENGTH: 78
Mapping created...
Single file converted to integers using the mapping
Training data created
Dataset size: 839870


In [43]:
# with open("training_data.pkl", "wb") as f:
#         pkl.dump([X, X_f, y, vocab_length, vocabulary], f)

with open("training_data.pkl", 'wb') as fo:  
    joblib.dump([X, X_f, y, vocab_length, vocabulary], fo)

In [37]:
X_f[:1]

array([[[0.87665929],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.87665929],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.87665929],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0.98119469],
        [0

In [9]:
mapped_song, vocab_length, vocabulary = mapping_data(single_file_song)

vocab length: 15


In [10]:
#ANALISIS DE CLASES MINORITARIAS PARA REDUCIR EL NUMERO DE LABELS
#Se analiza el vector de targets para revisar cuantos ejemplos hay de cada clase y luego eliminar los que tienen ocurrencias despreciables
#1. Convierte el vector de label en un integer
y_int = np.argmax(y, axis=1)

In [11]:
#2. Crea un diccionario con las frecuencias ocurrencias de cada clase en los labelst
import collections
counter = collections.Counter(y_int)
#y_counter = sorted(counter.items(), key=lambda x: x[1], reverse= True) 

In [13]:
counter[606]

40

In [18]:
#3. Guarda una lista con todos los labels que tienen menos de 10 ocurrencias en el dataset...

minority_clases = []
for i in range(len(counter)):
    if counter[i] <10:
        minority_clases.append(i)

print("There are {} classes with less than 10 examples".format(len(minority_clases)))
print("So we can reduce the classes from {0} to {1}".format(len(counter), len(counter) - len(minority_clases) ))

There are 1441 classes with less than 10 examples
So we can reduce the classes from 1809 to 368


In [15]:
len(counter) - len(minority_clases) #nro de labels que quedan 

368

In [161]:
##4. Eliminar todas los ejemplso que tengan clases con <10, que son 1441. Eso me deja 368 clases en vez de 1800...

def delete_examples(X, X_f, y, minority_clases):
    for i in range(len(minority_clases)):
        X = np.delete(X, r[i], 0)
        X_f = np.delete(X_f, r[i], 0)
        y = np.delete(y, r[i], 0)
    print("New shape of X data:", X.shape)
    print("New shape of y data:", y.shape)
    return X, X_f, y

In [162]:
X, X_f, y = delete_examples(X, X_f, y, minority_clases)

In [30]:
y_test = np.argmax(y, axis = 1)

for target in y_test:
    if target in minority_clases:
        print("yes")
    # if np.argmax(y_example, axis=0) in minority_clases:
    #     y = np.delete(y, y_example, 0)

yes
yes
yes
yes


In [23]:
y_x = np.array([[0,1,0,0,0], [0,0,0,0,1], [1,0,0,0,0], [0,1,0,0,0]])

In [24]:
y_x

array([[0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0]])

In [31]:
y[:3]

array([[0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0]])

In [28]:
np.argmax(y_x, axis = 1)

array([1, 4, 0, 1], dtype=int64)